In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# submission functionality
# ------------------------
import os, sys
sys.path.append(os.path.abspath("..")) # so src/ is on the path

import importlib
import submission_utils
importlib.reload(submission_utils) # force reload latest code

from submission_utils import save_submission
# ------------------------

# Load data
train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

# Target + ID
target = "Milk_Yield_L"
id_col = "Cattle_ID"

# Basic preprocessing
# (turn categories -> strings -> numeric encodings)
def preprocess(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype("category").cat.codes
    return df

train_prep = preprocess(train.drop(columns=[target]))
test_prep = preprocess(test)

y = train[target]
X = train_prep

# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)

    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    models.append(model)

print("\n==========================")
print(f"Average RMSE: {np.mean(fold_rmse):.4f}")
print("==========================\n")

# -----------------------
# Train final model on full data
# -----------------------
final_model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
final_model.fit(X, y)

# -----------------------
# Create submission
# -----------------------
test_preds = final_model.predict(test_prep)

submission = pd.DataFrame({
    id_col: test[id_col],
    target: test_preds
})

save_submission(submission, run_name="felipe_model")
print("submission.csv created!")


----- Fold 1 -----
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3988
[LightGBM] [Info] Number of data points in the train set: 168000, number of used features: 35
[LightGBM] [Info] Start training from score 15.587505
Fold 1 RMSE: 4.1518

----- Fold 2 -----
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004648 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3987
[LightGBM] [Info] Number of data points in the train set: 168000, number of used features: 35
[LightGBM] [Info] Start training from score 15.580781
Fold 2 RMSE: 4.1184

----- Fold 3 -----
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing wa

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# --- Reuse already-loaded train/test if they exist ---
# If not, load again
try:
    train
    test
    print("Reusing loaded train/test...")
except NameError:
    print("Loading data fresh...")
    train = pd.read_csv("../../data/cattle_data_train.csv")
    test = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"

def preprocess(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype("category").cat.codes
    return df

train_prep = preprocess(train.drop(columns=[target]))
test_prep = preprocess(test)

X = train_prep
y = train[target]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
xgb_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- XGBoost Fold {fold+1} -----")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = xgb.XGBRegressor(
        n_estimators=700,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",      # fast for large data
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)
    
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    xgb_models.append(model)

print("\n==========================")
print(f"XGBoost Average RMSE = {np.mean(fold_rmse):.4f}")
print("==========================\n")

# --- Train final model on all data ---
final_xgb = xgb.XGBRegressor(
    n_estimators=700,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist",
    random_state=42
)

final_xgb.fit(X, y)

# --- Submission ---
test_preds = final_xgb.predict(test_prep)

submission_xgb = pd.DataFrame({
    id_col: test[id_col],
    target: test_preds
})

save_submission(submission, run_name="felipe_model")
print("submission_xgb.csv created!")

Reusing loaded train/test...

----- XGBoost Fold 1 -----
Fold 1 RMSE: 4.1888

----- XGBoost Fold 2 -----
Fold 2 RMSE: 4.1517

----- XGBoost Fold 3 -----
Fold 3 RMSE: 4.1568

----- XGBoost Fold 4 -----


KeyboardInterrupt: 

In [ ]:
# If you haven't installed it yet:
# pip install catboost

from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# 5-Fold Cross-Validation for CatBoost
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cat_fold_rmse = []
cat_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- CatBoost Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=1000,
        learning_rate=0.05,
        depth=8,
        subsample=0.8,
        random_seed=42,
        verbose=False  # turn off big CatBoost logs
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    cat_fold_rmse.append(rmse)

    print(f"CatBoost Fold {fold+1} RMSE: {rmse:.4f}")
    cat_models.append(model)

print("\n==========================")
print(f"CatBoost Average RMSE: {np.mean(cat_fold_rmse):.4f}")
print("==========================\n")

# Train final CatBoost model on full data
final_cat_model = CatBoostRegressor(
    loss_function="RMSE",
    n_estimators=1000,
    learning_rate=0.05,
    depth=8,
    subsample=0.8,
    random_seed=42,
    verbose=False
)
final_cat_model.fit(X, y, verbose=False)

# Make submission
cat_test_preds = final_cat_model.predict(test_prep)

cat_submission = pd.DataFrame({
    id_col: test[id_col],
    target: cat_test_preds
})

cat_submission.to_csv("submission_catboost.csv", index=False)
print("submission_catboost.csv created!")


----- CatBoost Fold 1 -----
CatBoost Fold 1 RMSE: 4.1362

----- CatBoost Fold 2 -----
CatBoost Fold 2 RMSE: 4.1009

----- CatBoost Fold 3 -----
CatBoost Fold 3 RMSE: 4.1066

----- CatBoost Fold 4 -----
CatBoost Fold 4 RMSE: 4.1333

----- CatBoost Fold 5 -----
CatBoost Fold 5 RMSE: 4.1248

CatBoost Average RMSE: 4.1204

submission_catboost.csv created!
